In [2]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random
import os 
import numpy as np 
import tensorflow as tf
# Set a random seed for reproducibility
seed_value = 42
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
import pandas as pd 
from transformers import AutoTokenizer, TFAutoModel
import pandas as pd 
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Bidirectional
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

In [18]:
## load job models 

from tensorflow.keras.models import Model, load_model
import pickle
encoder_model = load_model('/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/processed_datasets_v2/models/job/encoder_model.h5')

# Save the decoder model
decoder_model = load_model('/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/processed_datasets_v2/models/job/decoder_model.h5')

with open('/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/processed_datasets_v2/models/job/tokenizer.pickle', 'rb') as handle:
    tokenizer_j_p = pickle.load(handle)

def decode_sequence_job(input_seq):
    # Encode the input sequence to get the internal states
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1 with only the start token
    target_seq = np.zeros((1, 1))
    start_token_index = tokenizer_j_p.word_index['start']  # Assuming you have a 'start' token
    target_seq[0, 0] = start_token_index

    # Sampling loop to generate sequence
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token and add its corresponding word to the decoded sequence
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer_j_p.index_word[sampled_token_index]
        if sampled_word != 'end':  # Assuming you have an 'end' token
            decoded_sentence += ' ' + sampled_word

        # Exit condition: either hit max length or find stop token
        if sampled_word == 'end' or len(decoded_sentence) > 50:
            stop_condition = True

        # Update the target sequence (of length 1) and states
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence


In [20]:
## load job test data 

tokenizer_j_e = AutoTokenizer.from_pretrained("bert-base-uncased")
model = TFAutoModel.from_pretrained("bert-base-uncased")

def get_embedding(text):
    inputs = tokenizer_j_e(text, return_tensors="tf", padding=True, truncation=True, max_length=512)
    outputs = model(inputs)
    return tf.reduce_mean(outputs.last_hidden_state, 1).numpy()[0]  # Mean pooling

def batch_encode(texts, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer_j_e(batch, return_tensors="tf", padding=True, truncation=True, max_length=512)
        outputs = model(inputs)
        batch_embeddings = tf.reduce_mean(outputs.last_hidden_state, 1).numpy()
        embeddings.extend(batch_embeddings)
    return embeddings


text_columns = ['Current Job', 'Next Job', 'Current Skills']
df_j = pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/processed_datasets_v2/original_data/jobs.csv")
df_user = df_j[df_j["Current Job"]=="data scientist"][0:1]

def predict_job(df_user):
    embeddings_dict_user = {}
    for col in text_columns:
        embeddings_dict_user[col] = batch_encode(df_user[col].tolist())

    feature_columns = ['Current Job', 'Current Skills']
    # Stack embeddings horizontally (axis=1)
    user_embedding = np.hstack((embeddings_dict_user[each] for each in feature_columns)) 

    user_embedding.shape
    user_embedding = user_embedding.reshape(1, len(feature_columns), -1)

    X_user_tensor = tf.convert_to_tensor(user_embedding, dtype=tf.float32)
    next_job = decode_sequence_job(X_user_tensor[0:1])
    return  next_job.replace("start ","").lstrip().rstrip()
predict_job(df_user)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

1/1 [==============================] - 0s 30ms/step


/var/folders/17/gws30jys63j9xx5l0mwr5r4r0000gq/T/ipykernel_72876/1249088265.py:33: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  user_embedding = np.hstack((embeddings_dict_user[each] for each in feature_columns))


1/1 [==============================] - 0s 30ms/step


'senior data scientist'

In [ ]:
################################################################################################################################################

In [11]:
#load skills models 

from tensorflow.keras.models import Model, load_model
import pickle
encoder_model_s = load_model('/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/processed_datasets_v2/models/skills/encoder_model.h5')

# Save the decoder model
decoder_model_s = load_model('/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/processed_datasets_v2/models/skills/decoder_model.h5')

with open('/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/processed_datasets_v2/models/skills/tokenizer.pickle', 'rb') as handle:
    tokenizer_s_p = pickle.load(handle)

def decode_sequence_skills(input_seq):
    # Encode the input sequence to get the internal states
    states_value = encoder_model_s.predict(input_seq)

    # Generate empty target sequence of length 1 with only the start token
    target_seq = np.zeros((1, 1))
    start_token_index = tokenizer_s_p.word_index['start']  # Assuming you have a 'start' token
    target_seq[0, 0] = start_token_index

    # Sampling loop to generate sequence
    stop_condition = False
    decoded_sequence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model_s.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer_s_p.index_word.get(sampled_token_index, '?')

        # Check for 'end' token or maximum length
        if sampled_word == 'end' or len(decoded_sequence) > 50:  # 'end' is your end-of-sequence token
            stop_condition = True
        else:
            decoded_sequence.append(sampled_word)

        # Update the target sequence (of length 1) and states
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    # Join the words with a comma
    decoded_sentence = ', '.join(decoded_sequence)
    return decoded_sentence


In [12]:
## load skills test data 

tokenizer_s_e = AutoTokenizer.from_pretrained("bert-base-uncased")
model = TFAutoModel.from_pretrained("bert-base-uncased")

def get_embedding(text):
    inputs = tokenizer_s_e(text, return_tensors="tf", padding=True, truncation=True, max_length=512)
    outputs = model(inputs)
    return tf.reduce_mean(outputs.last_hidden_state, 1).numpy()[0]  # Mean pooling

def batch_encode(texts, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer_s_e(batch, return_tensors="tf", padding=True, truncation=True, max_length=512)
        outputs = model(inputs)
        batch_embeddings = tf.reduce_mean(outputs.last_hidden_state, 1).numpy()
        embeddings.extend(batch_embeddings)
    return embeddings


text_columns = ['Current Job', 'Next Job', 'Current Skills','Next Skills']
df = pd.read_csv("//Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/processed_datasets_v2/original_data/skills.csv")
df_user = df[df["Current Job"]=="data scientist"][0:1]
embeddings_dict_user = {}
for col in text_columns:
    embeddings_dict_user[col] = batch_encode(df_user[col].tolist())

feature_columns = ['Current Job', 'Next Job','Current Skills']
# Stack embeddings horizontally (axis=1)
user_embedding = np.hstack((embeddings_dict_user[each] for each in feature_columns)) 

user_embedding.shape
user_embedding = user_embedding.reshape(1, len(feature_columns), -1)



X_user_tensor = tf.convert_to_tensor(user_embedding, dtype=tf.float32)
next_job = decode_sequence_skills(X_user_tensor[0:1])
next_job.replace("start ","")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

1/1 [==============================] - 0s 28ms/step


'python,  sql,  machine learning,  data analysis,  natural language processing,  algorithms,  statistical,  data visualization,  data mining,  text mining,  artificial intelligence,  visualization,  data science,  nosql'

In [49]:
current_job,final_job = "senior data analyst","big data engineer"
current_skills,next_skills="",""
df_j = pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/processed_datasets_v2/original_data/jobs.csv")
while current_job!=final_job:
    df_user = df_j[df_j["Current Job"]==current_job]
    df_user = df_user.sample(n=1)
    current_job = predict_job(df_user)
    print(current_job)


1/1 [==============================] - 0s 41ms/step


/var/folders/17/gws30jys63j9xx5l0mwr5r4r0000gq/T/ipykernel_72876/1249088265.py:33: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  user_embedding = np.hstack((embeddings_dict_user[each] for each in feature_columns))


1/1 [==============================] - 0s 37ms/step
data engineer
1/1 [==============================] - 0s 40ms/step
senior data engineer
1/1 [==============================] - 0s 38ms/step
data architect
1/1 [==============================] - 0s 36ms/step
data manager
1/1 [==============================] - 0s 34ms/step
director of analytics
1/1 [==============================] - 0s 35ms/step
chief data officer
1/1 [==============================] - 0s 53ms/step
senior data product manager
1/1 [==============================] - 0s 38ms/step
director of product management
1/1 [==============================] - 0s 37ms/step
chief product officer
1/1 [==============================] - 0s 34ms/step
big data engineer


In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_c_cleaned["full_description"])

In [47]:
df_j[df_j["Current Job"]=="senior data analyst"]

,Unnamed: 0,Current Job,Next Job,Current Skills
3,3,senior data analyst,data engineer,"bigquery, python, sql, airflow, problem solvin..."
75,76,senior data analyst,data engineer,"technology solutions, predictive analytics, py..."
93,95,senior data analyst,data engineer,"data processing, python, data strategy, comput..."
150,154,senior data analyst,data engineer,"data warehousing, data infrastructure, databas..."
398,442,senior data analyst,data engineer,"data processing, distributed file systems, sql..."
451,505,senior data analyst,data engineer,"python, sql, power bi, java, business intellig..."
492,555,senior data analyst,data engineer,"python, sql, c, data analysis, data visualizat..."
700,850,senior data analyst,data engineer,"computer science, data architecture, data qual..."
732,899,senior data analyst,data engineer,"python, power bi, sql, computer science, data ..."
749,926,senior data analyst,data engineer,"python, sql, computer science, problem solving..."


In [52]:
import pandas as pd
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
df_c_cleaned = pd.read_csv("/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/course_cleaned.csv")
df_c_cleaned=df_c_cleaned.rename(columns={"Unnamed: 0":"course_number"})
def calculate_course_score(course, skill_gap, weight_skill_match=1, weight_subscribers=0.001, weight_rating=0.5):
    # Calculate the proportion of skill gap covered by course
    skills_matched = sum(skill in course['skills_tagged'] for skill in skill_gap)
    skill_match_score = skills_matched / len(skill_gap) if skill_gap else 0

    # Factor in course popularity and rating
    #subscriber_score = course['num_subscribers'] * weight_subscribers if 'num_subscribers' in course else 0
    #rating_score = course['rating'] * weight_rating if 'rating' in course else 0

    # Calculate total score
    total_score = (skill_match_score * weight_skill_match) #+ subscriber_score + rating_score
    return total_score

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_c_cleaned["full_description"])

def calculate_similarity_score(skill_gap,df_c_cleaned):

    def preprocess_text(text):
        return " ".join(token.lemma_.lower() for token in nlp(text) if not token.is_punct and not token.is_stop)

    skill_gap_text = " ".join(skill_gap)
    # Vectorize the text

    skill_gap_vec = tfidf_vectorizer.transform([skill_gap_text])
    # Calculate cosine similarity
    cosine_similarities = cosine_similarity(skill_gap_vec, tfidf_matrix).flatten()
    df_c_cleaned["cosine_score"]=cosine_similarities
    return df_c_cleaned

def select_relevant_course(df_c_cleaned, skill_gap):
    # Apply the scoring function to each course
    df_c_cleaned['course_score'] = df_c_cleaned.apply(calculate_course_score, axis=1, skill_gap=skill_gap)
    df_c_cleaned = calculate_similarity_score(skill_gap,df_c_cleaned)
    # Sort courses based on the calculated score
    #df_c_cleaned = df_c_cleaned.sort_values(by='course_score', ascending=False)
    df_c_cleaned = df_c_cleaned.sort_values(by='cosine_score', ascending=False)
    # Select the top course if available
    top_course = df_c_cleaned.iloc[0] if not df_c_cleaned.empty else None
    top_10_course_number = df_c_cleaned["course_number"].iloc[0:10].tolist()
    top_10_course_number_score = {x:round(y,3) for x,y in zip(df_c_cleaned["course_number"].iloc[0:10].tolist(),df_c_cleaned["cosine_score"].iloc[0:10].tolist())}
    return top_course,top_10_course_number_score,top_10_course_number

top_course,top_10_course_number_score,top_10_course_number = select_relevant_course(df_c_cleaned,['python', 'power bi', 'sql'])


In [54]:

df_c_cleaned.iloc[top_10_course_number]

[3244, 3071, 3050, 1893, 2914, 922, 2757, 1624, 560, 2735]

,course_number,title,url,description,headline,num_subscribers,rating,num_reviews,num_quizzes,num_lectures,...,instructional_level,objectives,full_description,skills_tagged,price,course_title,course_url,course_display_name,course_score,cosine_score
3244,3244,power bi,https://www.udemy.com/course/power-bi-e/,p p p p,power bi,0,0.000000,0,0,169,...,Beginner Level,power bi,power bi p p p p power bi power bi power bi,{'power bi'},19.99,undefined,UnDefined,undefined,0.333333,0.792365
3071,3071,power bi,https://www.udemy.com/course/power-bi-idashboa...,p p p p p br p p power bi power bi p p br p p ...,power bi,7,5.000000,5,0,15,...,Intermediate Level,NaN,power bi p p p p p br p p power bi power bi p ...,{'power bi'},24.99,undefined,UnDefined,undefined,0.333333,0.786401
3050,3050,power bi,https://www.udemy.com/course/power-bi-ru/,p microsoft power bi power bi dax data visuali...,microsoft power bi,185,2.535770,6,0,25,...,All Levels,power bi,power bi p microsoft power bi power bi dax dat...,"{'power bi', 'data visualization'}",19.99,microsoft power bi,https://www.udemy.com/topic/microsoft-power-bi/,microsoft power bi,0.333333,0.743556
1893,1893,power skills mastering power bi practice tests,https://www.udemy.com/course/power-up-your-ski...,p ready power bi skills level look power skill...,master power bi beginner advanced unleash powe...,5,0.000000,0,6,0,...,All Levels,learners gain thorough understanding power bi ...,power skills mastering power bi practice tests...,"{'data modeling', 'power bi', 'data visualizat...",19.99,undefined,UnDefined,undefined,0.333333,0.608220
2914,2914,python sql,https://www.udemy.com/course/pythonsql/,p sql sql p p sql python python sqlite sql p p...,python sql python sql sql,114,3.737941,14,0,38,...,Beginner Level,sql,python sql p sql sql p p sql python python sql...,"{'sql', 'python', 'sqlite'}",19.99,sql,https://www.udemy.com/topic/sql/,sql,0.666667,0.587036
922,922,mastering microsoft power bi beginner advanced,https://www.udemy.com/course/mastering-microso...,p mastering microsoft power bi beginner advanc...,unlocking secrets data visualization transform...,2028,4.336264,49,0,50,...,All Levels,understanding power bi gain comprehensive unde...,mastering microsoft power bi beginner advanced...,"{'data visualization', 'analytics', 'visualiza...",49.99,undefined,UnDefined,undefined,0.333333,0.580480
2757,2757,microsoft power bi arabic,https://www.udemy.com/course/microsoft-power-b...,p data visualization microsoft power bi nbsp d...,microsoft power bi arabic,70,3.917178,34,0,44,...,All Levels,barcahrts,microsoft power bi arabic p data visualization...,"{'power bi', 'dataset', 'data visualization', ...",19.99,microsoft power bi,https://www.udemy.com/topic/microsoft-power-bi/,microsoft power bi,0.333333,0.565024
1624,1624,data science meets power bi transforming data ...,https://www.udemy.com/course/data-science-meet...,p welcome strong data science meets power bi t...,master python r power bi comprehensive data an...,74,4.779185,5,10,120,...,Intermediate Level,master integration python r power bi enhanced ...,data science meets power bi transforming data ...,"{'data science', 'algorithms', 'analytics', 's...",69.99,undefined,UnDefined,undefined,0.666667,0.551600
560,560,power bi bootcamp build real world power bi pr...,https://www.udemy.com/course/build-microsoft-p...,p gartner ranked microsoft leader gartner magi...,build real world microsoft power bi desktop pr...,2985,4.186971,165,0,56,...,All Levels,understand business intelligence workflow end end,power bi bootcamp build real world power bi pr...,"{'analytics', 'business intelligence', 'sql', ...",74.99,business intelligence,https://www.udemy.com/topic/business-intellige...,business intelligence,0.666667,0.530474
2735,2735,sql,https://www.udemy.com/course/cloom-sql-oracle/,p strong strong p p br p p p p ai p p br p p p...,sql,172,4.795263,39,0,5,...,All Levels,NaN,sql p strong strong p p br p p p p ai p p br p...,{'sql'},84.99,oracle sql,https://www.u

In [33]:
import json
Job_progression_dictionary_file_path = "/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data/processed_datasets/job_progression_dictionary.json"

with open(Job_progression_dictionary_file_path, 'r') as json_file:
    sorted_full_job_progression_dict_lower = json.load(json_file)

In [58]:
sorted_full_job_progression_dict_lower

{'ai architect': ['ai project manager', 'head of ai', 'chief ai officer'],
 'ai developer': ['senior ai developer',
  'ai architect',
  'ai project manager',
  'head of ai',
  'chief ai officer'],
 'ai project manager': ['head of ai', 'chief ai officer'],
 'big data architect': ['head of big data',
  'director of data engineering',
  'chief information officer'],
 'big data engineer': ['senior big data engineer',
  'big data architect',
  'head of big data',
  'director of data engineering',
  'chief information officer'],
 'business analysis manager': ['business intelligence analyst',
  'director of business analysis',
  'chief strategy officer',
  'data analyst',
  'data scientist'],
 'business analyst': ['senior business analyst',
  'business analysis manager',
  'business intelligence analyst',
  'director of business analysis',
  'chief strategy officer',
  'data analyst',
  'data scientist'],
 'business intelligence analyst': ['senior business intelligence analyst',
  'business i

In [37]:
sorted_full_job_progression_dict_lower

{'ai architect': ['ai project manager', 'head of ai', 'chief ai officer'],
 'ai developer': ['senior ai developer',
  'ai architect',
  'ai project manager',
  'head of ai',
  'chief ai officer'],
 'ai project manager': ['head of ai', 'chief ai officer'],
 'big data architect': ['head of big data',
  'director of data engineering',
  'chief information officer'],
 'big data engineer': ['senior big data engineer',
  'big data architect',
  'head of big data',
  'director of data engineering',
  'chief information officer'],
 'business analysis manager': ['business intelligence analyst',
  'director of business analysis',
  'chief strategy officer',
  'data analyst',
  'data scientist'],
 'business analyst': ['senior business analyst',
  'business analysis manager',
  'business intelligence analyst',
  'director of business analysis',
  'chief strategy officer',
  'data analyst',
  'data scientist'],
 'business intelligence analyst': ['senior business intelligence analyst',
  'business i